In [25]:
# Editor appearance set up & Load plot & Calculate DGCI

# Extend width of Jupyter Notebook Cell to the size of browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Import packages needed
import gc
import pickle
from tkinter import Tk
from tkinter.filedialog import askopenfilename, asksaveasfilename
import platform


from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib.widgets import RectangleSelector, PolygonSelector
import numpy as np
from skimage import io, draw
import mplcursors
import IPython.display as Disp
from ipywidgets import widgets
import cv2
from osgeo import gdal
import pandas as pd


# OS related settings
if platform.system() == 'Windows':
    print('Windows')
    # %matplotlib nbagg
    # Sometimes tk/qt will not let cells rerun after an ERROR occurs
#     %matplotlib tk
    %matplotlib qt
elif platform.system() == 'Darwin':
    print('macOS')
    Tk().withdraw()
    %matplotlib osx
elif platform == 'linux' or platform == 'linux2':
    print('Linux')
# This line of "print" must exist right after %matplotlib command, 
# otherwise JN will hang on the first import statement after this.
print('Interactive plot activated')


# Load image and print size & pre-process

# Use skimage to load multi-layer tiff file

image_file = askopenfilename(title='Load image file', initialdir='./output/field_image')
full_img = io.imread(image_file)
print("Original Image Shape: ", full_img.shape)

# Load GPS coordinate from file & Calculate pixel location
plot_loc = askopenfilename(title='Load plot location file', initialdir='./output/plot_location')
try:
    with open(plot_loc, 'rb') as f:
        plot_names = np.array(pickle.load(f))
        plot_vertices_gps = pickle.load(f)
except Exception as e:
    showerror(type(e).__name__, str(e))

# Calculating pixel location from GPS coordinate
ds = gdal.Open(image_file)
gt = ds.GetGeoTransform()
a, b, c, d, e, f = gt

plot_vertices = []
for i in range(len(plot_vertices_gps)):
    one_plot_vertices_gps = plot_vertices_gps[i]
    one_plot_vertices = []
    for vertex in one_plot_vertices_gps:
        xgeo, ygeo = vertex
        
        x = (xgeo - a) / b
        y = (ygeo - d) / f

        one_plot_vertices.append([x, y])
    one_plot_vertices = np.array(one_plot_vertices)
    one_plot_vertices = np.round(one_plot_vertices)
    one_plot_vertices = one_plot_vertices.astype(int)
    plot_vertices.append(one_plot_vertices)

print('Plot location loaded')

all_vertices = np.concatenate(plot_vertices, axis=0)
ul = np.amin(all_vertices, axis=0)
br = np.amax(all_vertices, axis=0)

img = full_img[ul[1]:br[1], ul[0]:br[0], :]
print("Trimmed Image Shape: ", img.shape)

h, w, d = img.shape

# Extract layers from the multilayer tiff file and do some adjustments
if d == 5:
    layer_RGB = img[:, :, 0:3]
    layer_IR = img[:, :, 3]
    layer_mask = img[:, :, 4]
    full_RGB = full_img[:, :, 0:3]
    # Change from 16 bit value to 0~1
    layer_RGB = layer_RGB / 65535
    full_RGB = full_RGB / 65535
    # Set background to black
    layer_RGB[np.where(layer_mask == 0)] = 0
    # RGB to HSV (H 0-1, S 0-1, V 0-255)
    layer_HSV = matplotlib.colors.rgb_to_hsv(layer_RGB)
    layer_hue = layer_HSV[:, :, 0]
    layer_saturation = layer_HSV[:, :, 1]
    layer_value = layer_HSV[:, :, 2]
    # Calculate Vegetation Index
    # Original formula
    # DGCI = ((layer_HSV[:, :, 0] - 60) / 60 + (1 - layer_HSV[:, :, 1]) + (1 - layer_HSV[:, :, 2]/255)) / 3
    layer_DGCI = (layer_hue * 360 / 60 + (1 - layer_saturation) + (1 - layer_value / 255)) / 3 
    print('DGCI calculated')
    layer_IR[np.where(layer_mask == 0)] = 0
elif d == 4:
    layer_RGB = img[:, :, 0:3]
    layer_IR = img[:, :, 0]
    layer_mask = img[:, :, 3]
    # Change from 16 bit value to 0~1
#     layer_RGB = layer_RGB / 65535
    # Set background to black
    layer_RGB[np.where(layer_mask == 0)] = 0
    layer_IR[np.where(layer_mask == 0)] = 0
elif d == 2:
    layer_IR = img[:, :, 0]
    layer_mask = img[:, :, 1]
    # Set background to black
    layer_IR[np.where(layer_mask == 0)] = 0


# Play sound when done
def done():
    print('\nDone')

    # Play sound when done
    
    if platform.system() == 'Windows':
        import winsound
        winsound.Beep(500, 1000)
    elif platform.system() == 'Darwin':
        import os
        os.system( "say Done" )
    elif platform == 'linux' or platform == 'linux2':
        import os
        os.system( "say Done" )
    # Reduce use of memory
    gc.collect()

done()

macOS
Interactive plot activated
Original Image Shape:  (12586, 40599, 5)
Plot location loaded
Trimmed Image Shape:  (3661, 31417, 5)
DGCI calculated

Done


In [ ]:
# Set hue range
def show_hue_area(x):
    hue_range = np.array(x)
    hue_mask[np.where((layer_hue>=hue_range[0]) * (layer_hue<=hue_range[1]))] = 1
    hue_mask[np.where(np.invert((layer_hue>=hue_range[0]) * (layer_hue<=hue_range[1])))] = 0
    red_masked_RGB = red * hue_mask * transparency + transformed_rgb * (1 - hue_mask * transparency)
    ax.imshow(red_masked_RGB)
    
    a, b, c = hue_mask.shape
    canopy = hue_mask[:, :, 0].sum()
    canopy_closure = canopy/(a*b)
    canopy_closure = "{:.2%}".format(canopy_closure)
    canopy_closure_text.value = canopy_closure
    
def change_transparency(x):
    transparency = x
    red_masked_RGB = red * hue_mask * transparency + transformed_rgb * (1 - hue_mask * transparency)
    ax.imshow(red_masked_RGB)

def save_hue_range(b):
    print('Hue range saved!')
    
color_map = 'gist_gray'

if layer_hue.max() <= 1:
    layer_hue = layer_hue * 360
hue_mask = np.zeros(full_RGB.shape)
transparency = 0.5
red = np.ones(hue_mask.shape) * (1, 0, 0)
transformed_rgb = full_RGB



fig, ax = plt.subplots(figsize=(5, 5))
flattened_layer_hue = layer_hue.flatten()
ax.hist(flattened_layer_hue, bins=360)
mean = np.mean(flattened_layer_hue)
var = np.var(flattened_layer_hue)
std = np.std(flattened_layer_hue)
print('Mean:', mean)
print('Variance:', var)
print('Standard Deviation', std)



default_hue_range = [60, 180]
button_layout = widgets.Layout(width='50%')
style = {'description_width': 'initial'}
hue_slider = widgets.FloatRangeSlider(value=default_hue_range, min=0, max=360, step=0.01, continuous_update=False, description='Hue Range', layout=widgets.Layout(width='70%'), style = {'description_width': 'initial'})
hue_interactive = widgets.interactive(show_hue_area, x=hue_slider)
transparency_slider = widgets.FloatSlider(value=0.5, min=0, max=1, continuous_update=False, description='Mask Transparency', readout_format='.1f', layout=widgets.Layout(width='70%'), style = {'description_width': 'initial'})
transparency_interactive = widgets.interactive(change_transparency, x=transparency_slider)
canopy_closure_text = widgets.Text(value='0', description='Canopy Closure', disabled=True, layout=widgets.Layout(width='40%'), style = {'description_width': 'initial'})


save_hue_button = widgets.Button(description='Save hue boundaries', layout=button_layout)

vbox_layout = widgets.Layout(align_items='center', width='50%')
buttons = widgets.VBox(children=[hue_slider, transparency_slider, canopy_closure_text, save_hue_button], layout=vbox_layout)

hbox_layout = widgets.Layout(align_items='center')
out = widgets.Output()
all_widgets = widgets.HBox(children=[out, buttons], layout=hbox_layout)
display(all_widgets)

save_hue_button.on_click(save_hue_range)



with out:
    fig, ax = plt.subplots(figsize=(5, 5))
    fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9)
    show_hue_area(default_hue_range)
    plt.show()

In [15]:
# Show indices of plots

def show_RGB(b):
    global cb
    if cb is not None:
        cb.remove()
        cb = None
    ax.imshow(layer_RGB)
    
def show_IR(b):
    global cb
    if cb is not None:
        cb.remove()
    axim = ax.imshow(layer_IR, cmap=plt.get_cmap(color_map))
    cb = fig.colorbar(axim)
    
def show_DGCI_image(b):
    global cb
    if cb is not None:
        cb.remove()
    axim = ax.imshow(layer_DGCI, cmap=plt.get_cmap(color_map))
    cb = fig.colorbar(axim)
    
def show_temp(b):
    if b.description == 'Show Average Temperature':
        for i in range(len(plot_names)):
            one_plot_vertices = plot_vertices[i]
            one_plot_vertices_transformed = one_plot_vertices - ul
            
            polygon = patches.Polygon(one_plot_vertices_transformed, True, facecolor = matplotlib.colors.to_rgba('red', 0.05), edgecolor=matplotlib.colors.to_rgba('orange', 0.5))
            ax.add_patch(polygon)
            text_loc = np.mean(one_plot_vertices_transformed, 0)
            axtx = ax.text(text_loc[0], text_loc[1], plot_names[i] + '\n' + str(avg_temps[i]) + '℃', ha='center', va='center')
            
        b.description = 'Hide Average Temperature'
    
    elif b.description == 'Hide Average Temperature':
        ax.patches.clear()
        ax.texts.clear()
        b.description = 'Show Average Temperature'
        plt.show()
    
def show_DGCI(b):
    if b.description == 'Show Average DGCI': 
        for i in range(len(plot_names)):
            one_plot_vertices = plot_vertices[i]
            one_plot_vertices_transformed = one_plot_vertices - ul
            
            polygon = patches.Polygon(one_plot_vertices_transformed, True, facecolor = matplotlib.colors.to_rgba('red', 0.05), edgecolor=matplotlib.colors.to_rgba('orange', 0.5))
            ax.add_patch(polygon)
            text_loc = np.mean(one_plot_vertices_transformed, 0)
            axtx = ax.text(text_loc[0], text_loc[1], plot_names[i] + '\n' + str(avg_DGCIs[i]), ha='center', va='center')
            
        b.description = 'Hide Average DGCI'
    
    elif b.description == 'Hide Average DGCI':
        ax.patches.clear()
        ax.texts.clear()
        b.description = 'Show Average DGCI'
        plt.show()



# Calculate avg temp & DGCI for each plot
avg_temps = []
max_temps = []
min_temps = []
avg_DGCIs = []
max_DGCIs = []
min_DGCIs = []
avg_RGBs = []
avg_HSVs = []

avg_temps_hue_restrict = []
max_temps_hue_restrict = []
min_temps_hue_restrict = []
avg_DGCIs_hue_restrict = []
max_DGCIs_hue_restrict = []
min_DGCIs_hue_restrict = []
avg_RGBs_hue_restrict = []
avg_HSVs_hue_restrict = []

canopy_closures = []
for i in range(len(plot_names)):
    one_plot_vertices = plot_vertices[i]
    one_plot_vertices_transformed = one_plot_vertices - ul
    
    rr, cc = draw.polygon(one_plot_vertices_transformed[:, 1], one_plot_vertices_transformed[:, 0], layer_IR.shape)
    plot_mask = np.zeros(layer_IR.shape, np.float)
    plot_mask[rr, cc] = 1
    
    inds = np.where(plot_mask == 1)
    
    avg_IR = np.average(layer_IR[inds])
    avg_temps.append(avg_IR)
    
    max_IR = np.max(layer_IR[inds])
    min_IR = np.min(layer_IR[inds])
    max_temps.append(max_IR)
    min_temps.append(min_IR)
    
    if d != 2:
        avg_IR = np.average(layer_IR[inds])
        avg_temp = np.round(avg_IR * 0.04 - 273.15, 2)
        avg_temps.append(avg_temp)

        max_IR = np.max(layer_IR[inds])
        min_IR = np.min(layer_IR[inds])
        max_temp = np.round(max_IR * 0.04 - 273.15, 2)
        min_temp = np.round(min_IR * 0.04 - 273.15, 2)
        max_temps.append(max_temp)
        min_temps.append(min_temp)
        
        avg_DGCI = np.average(layer_DGCI[inds])
        avg_DGCI = np.round(avg_DGCI, 2)
        avg_DGCIs.append(avg_DGCI)

        max_DGCI = np.max(layer_DGCI[inds])
        min_DGCI = np.min(layer_DGCI[inds])
        max_DGCIs.append(max_DGCI)
        min_DGCIs.append(min_DGCI)

        avg_RGB = np.mean(layer_RGB[inds], axis=0)
        avg_RGB = np.round(avg_RGB, 2)
        avg_RGBs.append(avg_RGB)

        avg_HSV = np.mean(layer_HSV[inds], axis=0)
        avg_HSV = np.round(avg_HSV, 2)
        avg_HSVs.append(avg_HSV)
    
    
    # Apply hue mask
    if d == 5:
        mask_hue_restrict = plot_mask * hue_mask[:, :, 0]
        inds = np.where(mask_hue_restrict == 1)

        avg_IR = np.average(layer_IR[inds])
        avg_temp = np.round(avg_IR * 0.04 - 273.15, 2)
        avg_temps_hue_restrict.append(avg_temp)

        max_IR = np.max(layer_IR[inds])
        min_IR = np.min(layer_IR[inds])
        max_temp = np.round(max_IR * 0.04 - 273.15, 2)
        min_temp = np.round(min_IR * 0.04 - 273.15, 2)
        max_temps_hue_restrict.append(max_temp)
        min_temps_hue_restrict.append(min_temp)

        avg_DGCI = np.average(layer_DGCI[inds])
        avg_DGCI = np.round(avg_DGCI, 2)
        avg_DGCIs_hue_restrict.append(avg_DGCI)

        max_DGCI = np.max(layer_DGCI[inds])
        min_DGCI = np.min(layer_DGCI[inds])
        max_DGCIs_hue_restrict.append(max_DGCI)
        min_DGCIs_hue_restrict.append(min_DGCI)

        avg_RGB = np.mean(layer_RGB[inds], axis=0)
        avg_RGB = np.round(avg_RGB, 2)
        avg_RGBs_hue_restrict.append(avg_RGB)

        avg_HSV = np.mean(layer_HSV[inds], axis=0)
        avg_HSV = np.round(avg_HSV, 2)
        avg_HSVs_hue_restrict.append(avg_HSV)

        cnp_cls = mask_hue_restrict.sum()/plot_mask.sum()
        cnp_cls = "{:.2%}".format(cnp_cls)
        canopy_closures.append(cnp_cls)
    
avg_temps = np.array(avg_temps)
avg_DGCIs = np.array(avg_DGCIs)
avg_RGBs = np.array(avg_RGBs)
avg_HSVs = np.array(avg_HSVs)

if d == 5:
    avg_temps_hue_restrict = np.array(avg_temps_hue_restrict)
    avg_DGCIs_hue_restrict = np.array(avg_DGCIs_hue_restrict)
    avg_RGBs_hue_restrict = np.array(avg_RGBs_hue_restrict)
    avg_HSVs_hue_restrict = np.array(avg_HSVs_hue_restrict)
    


button_layout = widgets.Layout(width='50%')
show_RGB_button = widgets.Button(description='Show RGB Image', layout=button_layout)
show_IR_button = widgets.Button(description='Show IR Image', layout=button_layout)
show_DGCIImage_button = widgets.Button(description='Show DGCI Image', layout=button_layout)
show_temp_button = widgets.Button(description='Show Average Temperature', layout=button_layout)
show_DGCI_button = widgets.Button(description='Show Average DGCI', layout=button_layout)

vbox_layout = widgets.Layout(align_items='center', width='50%')
buttons = widgets.VBox(children=[show_RGB_button, show_IR_button, show_DGCIImage_button, show_temp_button, show_DGCI_button], layout=vbox_layout)

out = widgets.Output()
hbox_layout = widgets.Layout(align_items='center')
all_widgets = widgets.HBox(children=[out, buttons], layout=hbox_layout)
display(all_widgets)

show_RGB_button.on_click(show_RGB)
show_IR_button.on_click(show_IR)
show_DGCIImage_button.on_click(show_DGCI_image)
show_temp_button.on_click(show_temp)
show_DGCI_button.on_click(show_DGCI)

with out:
    fig, ax = plt.subplots(figsize=(5, 5))
    fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9)
    if d != 2:
        ax.imshow(layer_RGB)
    else:
        ax.imshow(layer_IR)
    cb = None
    plt.show()

In [7]:
# Save to table

if d == 5:
    df = pd.DataFrame(data=np.column_stack((plot_names, avg_temps, avg_temps_hue_restrict, max_temps, max_temps_hue_restrict, min_temps, min_temps_hue_restrict, avg_DGCIs, avg_DGCIs_hue_restrict, max_DGCIs, max_DGCIs_hue_restrict, min_DGCIs, min_DGCIs_hue_restrict, canopy_closures, avg_RGBs, avg_RGBs_hue_restrict, avg_HSVs, avg_HSVs_hue_restrict)),
                  columns=['Plot Name', 'Avg Temp', 'Avg Temp (hue restrict)', 'Max Temp', 'Max Temp (hue restrict)', 'Min Temp', 'Min Temp (hue restrict)', 'Avg DGCI', 'Avg DGCI (hue restrict)', 'Max DGCI', 'Max DGCI (hue restrict)', 'Min DGCI', 'Min DGCI (hue restrict)', 'Canopy Closure', 'Avg R', 'Avg G', 'Avg B', 'Avg R (hue restrict)', 'Avg G (hue restrict)', 'Avg B (hue restrict)', 'Avg H', 'Avg S', 'Avg V', 'Avg H (hue restrict)', 'Avg S (hue restrict)', 'Avg V (hue restrict)'])
elif d == 4:
    df = pd.DataFrame(data=np.column_stack((plot_names, avg_temps, max_temps, min_temps, avg_DGCIs, max_DGCIs, min_DGCIs, avg_RGBs, avg_HSVs)),
                  columns=['Plot Name', 'Avg Temp', 'Max Temp', 'Min Temp', 'Avg DGCI', 'Max DGCI', 'Min DGCI', 'Avg R', 'Avg G', 'Avg B', 'Avg H', 'Avg S', 'Avg V',])
elif d == 2:
    df = pd.DataFrame(data=np.column_stack((plot_names, avg_temps, max_temps, min_temps)),
                  columns=['Plot Name', 'Avg Temp', 'Max Temp', 'Min Temp'])
print(df)


def ask_to_save():
    fn = image_file.split('/')[-2] + '_' + image_file.split('/')[-1].split('.')[0]
    file_name = asksaveasfilename(filetypes=[('csv', '*.csv')], title='Save Indices', initialfile=fn+'_indices', initialdir='./output/')
    if not file_name:
        return
    if not file_name.endswith('.csv'):
        file_name += '.csv'
        
    try:
        df.to_csv(file_name)
        print('Indices saved to', file_name)
    except Exception as e:
        showerror(type(e).__name__, str(e))


# Tk().withdraw()
ask_to_save()

done()

  Plot Name   Avg Temp   Max Temp   Min Temp
0        pa  33.809673  57.469986  28.265907
1        pb  32.077557  57.878296   25.26109

Done


In [5]:
fig, ax = plt.subplots(figsize=(5, 5))
flattened_layer_IR = layer_IR.flatten()
ax.hist(flattened_layer_IR, bins=360)

(array([1.00000e+00, 1.00000e+00, 1.00000e+00, 6.00000e+00, 1.80000e+01,
        3.10000e+01, 7.30000e+01, 1.07000e+02, 1.80000e+02, 2.48000e+02,
        3.34000e+02, 4.80000e+02, 6.17000e+02, 7.72000e+02, 9.53000e+02,
        1.11200e+03, 1.30300e+03, 1.42800e+03, 1.49300e+03, 1.59300e+03,
        1.68900e+03, 1.70100e+03, 1.86400e+03, 1.86100e+03, 1.87300e+03,
        1.93600e+03, 1.87300e+03, 1.95800e+03, 2.14900e+03, 2.48800e+03,
        3.27700e+03, 4.67600e+03, 6.55400e+03, 8.94500e+03, 1.06680e+04,
        1.13460e+04, 1.16450e+04, 1.21370e+04, 1.23700e+04, 1.26280e+04,
        1.38190e+04, 1.50520e+04, 1.61640e+04, 1.76190e+04, 1.90880e+04,
        2.10110e+04, 2.29090e+04, 2.49880e+04, 2.71850e+04, 3.02990e+04,
        3.33680e+04, 3.65460e+04, 4.08710e+04, 4.46570e+04, 4.91870e+04,
        5.51060e+04, 6.04570e+04, 6.72790e+04, 7.19680e+04, 7.69620e+04,
        8.22100e+04, 8.77130e+04, 9.26540e+04, 9.91520e+04, 1.04319e+05,
        1.10612e+05, 1.16147e+05, 1.18703e+05, 1.20